In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ast

In [2]:
df = pd.read_excel('dados_brutos.xlsx')

In [3]:
df.shape

(2449, 13)

In [4]:
df.columns

Index(['Unnamed: 0', 'user', 'organization_id', 'org_name', 'org_size',
       'area_name', 'position_name', 'day', 'path', 'path_actions',
       'time_path', 'cookie', 'session'],
      dtype='object')

In [5]:
# fig = plt.figure(figsize=(40,15))
# plt.hist(df['position_name'])
# plt.show()

In [6]:
# df['org_name'].value_counts()

In [7]:
#como pegar os itens unicos sem usar set!!!
# user_unico = {}
# for k,v in users.items():
#     if v not in user_unico.keys():
#         user_unico[v] = 1
#     else:
#         user_unico[v] += 1

In [8]:
orgs = df['organization_id']
users = df['user']
org_u = set(orgs)
dic_userPorOrg = {}
for i,org in enumerate(org_u):
    df_aux = df.loc[(df["organization_id"] == org),:]
    user_aux = df_aux["user"]
    dic_userPorOrg[org] = len(set(user_aux))


In [9]:
dic_userPorOrg['4adb5f9b-f50a-4957-a7e3-94c3452d877c']

35

In [10]:
organizations = []
n_users = []
for k,v in dic_userPorOrg.items():
    organizations.append(k)
    n_users.append(v)

df_userPorOrg = pd.DataFrame({
    'organization_id': organizations,
    'N_users': n_users
})

In [11]:
df_userPorOrg = df_userPorOrg.sort_values(ascending=False, by='N_users').reset_index(drop=True)

In [12]:
df_userPorOrg

,organization_id,N_users
0,4f3e63d2-43f8-4886-addc-9ce039eab5f1,53
1,4adb5f9b-f50a-4957-a7e3-94c3452d877c,35
2,d3400673-a5b2-4914-bf57-ce98dc710603,27
3,c81fa8b0-52c0-4a31-abb7-6776a76cc9c8,27
4,347ceb13-8656-4ed0-b080-c6cfb5528b25,26
...,...,...
173,7f7ddce9-efb2-445d-b475-8942a226b6cd,1
174,c873af48-b59f-48af-a0a9-70171c1144f6,1
175,23cc5646-d288-455a-8309-a0710a29f7a0,1
176,59aae588-31e1-447f-a132-516d7e4dd1aa,1


In [13]:
dic_pathPorOrg = {}
for columns,rows in df_userPorOrg.head(20).items():
    if columns == 'organization_id':
        for org in rows:
            path_aux = df.loc[(df["organization_id"] == org),(["path_actions"])]
            paths_list = []
            for paths in (path_aux.values):
                lista = (ast.literal_eval(paths[0]))
                for e in set(lista):
                    paths_list.append(e)
            dic_pathPorOrg[org] = (set(paths_list))

In [14]:
organizations2 = []
paths = []
for k,v in dic_pathPorOrg.items():
    organizations2.append(k)
    paths.append(v)

df_pathPorOrg = pd.DataFrame({
    'organization_id': organizations2,
    'Paths': paths
})

In [15]:
df_pathPorOrg

,organization_id,Paths
0,4f3e63d2-43f8-4886-addc-9ce039eab5f1,"{clickNotification-hall, handleViewDesafioSRG-..."
1,4adb5f9b-f50a-4957-a7e3-94c3452d877c,"{clickNotification-hall, handleViewDesafioSRG-..."
2,d3400673-a5b2-4914-bf57-ce98dc710603,"{handleViewDesafioSRG-hall, click em módulos-l..."
3,c81fa8b0-52c0-4a31-abb7-6776a76cc9c8,"{clickNotification-hall, handleViewDesafioSRG-..."
4,347ceb13-8656-4ed0-b080-c6cfb5528b25,"{clickNotification-hall, handleViewDesafioSRG-..."
5,9e3e7a95-c6f5-4b6f-b189-8f33e45ee425,"{clickNotification-hall, handleViewDesafioSRG-..."
6,abba8235-15bd-4285-90b4-104e9308013e,"{clickNotification-hall, handleViewDesafioSRG-..."
7,50ca2381-2d91-4d7e-8300-bb625d5cb97f,"{clickNotification-hall, handleViewDesafioSRG-..."
8,6019ee26-09c3-422e-8c25-0abe878d1a73,"{clickNotification-hall, handleViewDesafioSRG-..."
9,c8c88852-b101-4b47-aaca-4cee45368c9f,"{clickNotification-hall, handleViewDesafioSRG-..."
